In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame({'col1':np.random.choice(list('abcde'),100),
                  'col2':np.random.choice(list('xyz'),100)}, dtype='category')
df1 = pd.DataFrame({'col1':np.random.choice(list('abcde'),100),
                   'col2':np.random.choice(list('xyz'),100)}, dtype='category')

dfa = pd.get_dummies(df)
dfb = pd.get_dummies(df1)
dfa.corrwith(dfb)

col1_a   -0.268093
col1_b    0.250958
col1_c   -0.061957
col1_d   -0.106948
col1_e    0.066750
col2_x   -0.128103
col2_y   -0.011029
col2_z    0.035407
dtype: float64

In [6]:
df = pd.DataFrame({'cat': pd.Categorical(['a', 'a', 'a', 'b', 'b', 'c'])})
df2 = pd.DataFrame({'cat': [1, 1, 1, 2, 2, 3]})

print(df.info())

print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
cat    6 non-null category
dtypes: category(1)
memory usage: 110.0 bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
cat    6 non-null int64
dtypes: int64(1)
memory usage: 128.0 bytes
None


In [8]:
raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'],
        'last_name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze'],
        'sex': ['male', 'female', 'male', 'female', 'female']}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'sex'])

df_sex = pd.get_dummies(df['sex'])


df_new = df.join(df_sex)
df_new

del df_new['sex']
df_new

,first_name,last_name,female,male
0,Jason,Miller,0,1
1,Molly,Jacobson,1,0
2,Tina,Ali,0,1
3,Jake,Milner,1,0
4,Amy,Cooze,1,0


In [4]:
df_aluno = pd.read_json("thiago/Aluno.json")
df_Caracterizacao = pd.read_json("thiago/Caracterizacao.json")
df_CursoCampus = pd.read_json("thiago/CursoCampus.json")
df_Raca = pd.read_json("thiago/Raca.json")
df_SituacaoMatricula = pd.read_json("thiago/SituacaoMatricula.json")
df_EstadoCivil = pd.read_json("thiago/EstadoCivil.json")

#merge entre as tabelas 
df = pd.merge(df_Caracterizacao, df_aluno, left_on="aluno_id", right_on="id", suffixes=["_L", "_R"])
df = pd.merge(df, df_CursoCampus, left_on="curso_campus_id", right_on="id", suffixes=["_L", "_curso"])
df = pd.merge(df, df_Raca, left_on="raca_id", right_on="id", suffixes=["_curso", "_raca"])
df = pd.merge(df, df_EstadoCivil, left_on="estado_civil_id", right_on="id")
df = pd.merge(df, df_SituacaoMatricula, left_on="situacao_id", right_on="id", suffixes=["_civil", "_situacao"])

#filtrando dados
df = df[['renda_bruta_familiar', 
         'descricao_curso', 
         'descricao_raca', 
         'descricao_civil', 
         'descricao_situacao'
       ]]

#filtrando usuários q tenham renda bruta maior que 20k
df = df[(df.renda_bruta_familiar <= 20000) & (df.renda_bruta_familiar > 0)]

bins = [0, 1874, 3748, 9370, 18740, 20000]
group_names = ['E', 'D', 'C', 'B', 'A']

df['categorias'] = pd.cut(df['renda_bruta_familiar'], bins, labels=group_names)

df.head

<bound method NDFrame.head of       renda_bruta_familiar                                    descricao_curso  \
1                    800.0  Técnico de Nível Médio em Segurança do Trabalh...   
2                   2900.0  Técnico de Nível Médio em Segurança do Trabalh...   
3                    710.0  Técnico de Nível Médio em Segurança do Trabalh...   
4                   1000.0  Técnico de Nível Médio em Segurança do Trabalh...   
5                    626.0  Técnico de Nível Médio em Segurança do Trabalh...   
6                    720.0  Técnico de Nível Médio em Segurança do Trabalh...   
7                    750.0  Técnico de Nível Médio em Segurança do Trabalh...   
8                   1250.0  Técnico de Nível Médio em Segurança do Trabalh...   
9                    724.0  Técnico de Nível Médio em Segurança do Trabalh...   
10                   726.0  Técnico de Nível Médio em Segurança do Trabalh...   
12                   788.0  Técnico de Nível Médio em Segurança do Trabalh...  

In [15]:
pd.crosstab(df.categorias, df.descricao_civil, margins=True)

descricao_civil,Casado(a),Divorciado(a),Não declarado,Solteiro(a),União Estável,Viúvo(a),All
categorias,,,,,,,
E,458,138,7,1732,132,2,2469
D,196,63,3,460,30,2,754
C,64,16,1,168,17,0,266
B,4,4,0,10,0,0,18
A,0,0,0,2,0,0,2
All,722,221,11,2372,179,4,3509


In [24]:
pd.crosstab(df.descricao_situacao, [df.categorias, df.descricao_civil, df.descricao_curso, df.descricao_raca], margins=True).apply(lambda r: r/len(df)*100, axis=1)

categorias                                                                                E  \
descricao_civil                                                                   Casado(a)   
descricao_curso                 Licenciatura em Espanhol [2006-2011] - Campus Natal-Central   
descricao_raca                                                                      Amarela   
descricao_situacao                                                                            
Aguardando Colação de Grau                                                     0.0            
Cancelado                                                                      0.0            
Cancelamento Compulsório                                                       0.0            
Concludente                                                                    0.0            
Concluído                                                                      0.0            
Evasão                                                                         0.0            
Formado                                                                        0.0            
Jubilado                                                                       0.0            
Matriculado                                                                    0.0            
Matrícula Vínculo Institucional                                                0.0            
Trancado                                                                       0.0            
Trancado Voluntariamente                                                       0.0            
Transferido Externo                                                            0.0            
Transferido Interno                                                            0.0            
All                                                                            0.0            

categorias                                                                  \
descricao_civil                                                              
descricao_curso                                                              
descricao_raca                     Branca Indígena Não declarado     Parda   
descricao_situacao                                                           
Aguardando Colação de Grau       0.000000      0.0           0.0  0.000000   
Cancelado                        0.000000      0.0           0.0  0.000000   
Cancelamento Compulsório         0.000000      0.0           0.0  0.000000   
Concludente                      0.000000      0.0           0.0  0.000000   
Concluído                        0.000000      0.0           0.0  0.000000   
Evasão                           0.028498      0.0           0.0  0.000000   
Formado                          0.085494      0.0           0.0  0.085494   
Jubilado                         0.000000      0.0           0.0  0.000000   
Matriculado                      0.000000      0.0           0.0  0.000000   
Matrícula Vínculo Institucional  0.000000      0.0           0.0  0.000000   
Trancado                         0.000000      0.0           0.0  0.000000   
Trancado Voluntariamente         0.000000      0.0           0.0  0.000000   
Transferido Externo              0.000000      0.0           0.0  0.000000   
Transferido Interno              0.000000      0.0           0.0  0.000000   
All                              0.113993      0.0           0.0  0.085494   

categorias                                 \
descricao_civil                             
descricao_curso                             
descricao_raca                      Preta   
descricao_situacao                          
Aguardando Colação de Grau       0.000000   
Cancelado                        0.000000   
Cancelamento Compulsório         0.000000   
Concludente                      0.000000   
Concluído                        0.000000   
Evasão                           0.000000   
Formado                          0.085494  

In [5]:
#http://nbviewer.jupyter.org/github/esafak/mca/blob/master/docs/mca-BurgundiesExample.ipynb
#https://pypi.python.org/pypi/mca/1.0
import mca
import pandas as pd
import numpy as np

np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
pd.set_option('display.precision', 5)
pd.set_option('display.max_columns', 25)

In [ ]:
#outra representacao https://arxiv.org/pdf/1410.7404.pdf Maximally Informative Hierarchical Representations
#of High-Dimensional Data

In [7]:
#filtrando dados
df = df[['categorias', 
         'descricao_curso', 
         'descricao_raca', 
         'descricao_civil', 
         'descricao_situacao'
       ]]

df.head

<bound method NDFrame.head of      categorias                                    descricao_curso  \
1             E  Técnico de Nível Médio em Segurança do Trabalh...   
2             D  Técnico de Nível Médio em Segurança do Trabalh...   
3             E  Técnico de Nível Médio em Segurança do Trabalh...   
4             E  Técnico de Nível Médio em Segurança do Trabalh...   
5             E  Técnico de Nível Médio em Segurança do Trabalh...   
6             E  Técnico de Nível Médio em Segurança do Trabalh...   
7             E  Técnico de Nível Médio em Segurança do Trabalh...   
8             E  Técnico de Nível Médio em Segurança do Trabalh...   
9             E  Técnico de Nível Médio em Segurança do Trabalh...   
10            E  Técnico de Nível Médio em Segurança do Trabalh...   
12            E  Técnico de Nível Médio em Segurança do Trabalh...   
13            E  Técnico de Nível Médio em Segurança do Trabalh...   
14            E  Técnico de Nível Médio em Segurança do Trab

In [11]:
#pd.get_dummies(df).info()
df.tail()


,categorias,descricao_curso,descricao_raca,descricao_civil,descricao_situacao
3658,E,Licenciatura em Espanhol [2012] - Campus Natal...,Parda,Casado(a),Matrícula Vínculo Institucional
3659,E,Licenciatura em Espanhol [2012] - Campus Natal...,Parda,Casado(a),Matrícula Vínculo Institucional
3660,E,Licenciatura em Letras Espanhol EaD - Campus EaD,Branca,Casado(a),Matrícula Vínculo Institucional
3661,D,Licenciatura em Espanhol [2006-2011] - Campus ...,Preta,Casado(a),Matrícula Vínculo Institucional
3662,B,Licenciatura em Letras Espanhol EaD - Campus EaD,Parda,Casado(a),Aguardando Colação de Grau


In [14]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
#X = enc.fit_transform( df ).toarray()
X = pd.get_dummies(df)

#print(X)

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

print(X_pca)

[[-0.8308 -0.1052 -0.4092]
 [-0.2215  0.5841 -0.0275]
 [-0.8308 -0.1052 -0.4092]
 ..., 
 [ 0.9068  0.2974 -0.1438]
 [ 0.9015  1.1983  0.1980]
 [ 0.0736  1.3030  0.3960]]


In [15]:
df.to_csv("dados_para_R.csv")